# TCPIP
大多数连接都是可靠的TCP连接。创建TCP连接时，主动发起连接的叫客户端，被动响应连接的叫服务器。
## client

In [5]:
# client
import socket

# 创建一个socket:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# 建立连接:
s.connect(('www.sina.com.cn', 80))
s.send(b'GET / HTTP/1.1\r\nHost: www.sina.com.cn\r\nConnection: close\r\n\r\n')
# 接收数据:
buffer = []
while True:
    # 每次最多接收1k字节:
    d = s.recv(1024)
    if d:
        buffer.append(d)
    else:
        break
data = b''.join(buffer)
s.close()

header, html = data.split(b'\r\n\r\n', 1)
print(header.decode('utf-8'))
# # 把接收的数据写入文件:
# with open('sina.html', 'wb') as f:
#     f.write(html)

HTTP/1.1 302 Moved Temporarily
Server: nginx
Date: Sun, 18 Aug 2019 06:47:12 GMT
Content-Type: text/html
Content-Length: 154
Connection: close
Location: https://www.sina.com.cn/
X-Via-CDN: f=edge,s=ctc.nanjing.ha2ts4.92.nb.sinaedge.com,c=219.233.222.198;
X-Via-Edge: 1566110832595c6dee9db7c5e66ca7b3b2bf2


In [ ]:
# server
import socket
import threading
import time

def sever():
    '''服务器程序'''
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('127.0.0.1', 9999))    # 监听端口
    s.listen(5)
    print('Waiting for connection...')

    while True:
        sock, addr = s.accept()    # 接受一个新连接
        t = threading.Thread(target=tcplink, args=(sock, addr))    # 创建新线程来处理TCP连接
        t.start()

def tcplink(sock, addr):
    '''收发程序'''
    print('Accept new connection from %s:%s...' % addr)
    sock.send(b'Welcome!')
    while True:
        data = sock.recv(1024)
        time.sleep(1)
        if not data or data.decode('utf-8') == 'exit':
            break
        sock.send(('Hello, %s!' % data.decode('utf-8')).encode('utf-8'))
    sock.close()
    print('Connection from %s:%s closed.' % addr)

if __name__ == '__main__':
    sever()